# Spatial regression

In [ ]:
gdf_name = "results_pcparis.gpkg" 

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
os.environ['USE_PYGEOS'] = '0'
import pysal.lib
import helpers as hs
from importlib import reload
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%cd ../data
gdf = gpd.read_file(gdf_name, layer="cool")

In [ ]:
gdfs = gdf.copy()
gdfs["%_soc.minimum"] = (gdfs["Men_pauv"]/gdfs["Ind"])*100
gdfs["%_>_65"] = ((gdfs["Ind_65_79"] +gdfs["Ind_80p"])/gdfs["Ind"])*100
gdfs["%_<_17"] = ((gdfs["Ind_0_3"] +gdfs["Ind_4_5"] + gdfs["Ind_6_10"] + gdfs["Ind_11_17"])/gdfs["Ind"])*100
gdfs["%_<_bat_45"] = (gdfs["Log_av45"]/(gdfs["Log_av45"] +gdfs["Log_45_70"] + gdfs["Log_70_90"] + gdfs["Log_ap90"] + gdfs["Log_inc"]))*100
gdfs["%_>_bat_90"] = (gdfs["Log_ap90"]/(gdfs["Log_av45"] +gdfs["Log_45_70"] + gdfs["Log_70_90"] + gdfs["Log_ap90"] + gdfs["Log_inc"]))*100
gdfs["%_residences"] = (gdfs["Men_coll"]/(gdfs["Men_coll"] +gdfs["Men_mais"]))*100
gdfs["mean_income"] = gdfs["Ind_snv"]/gdfs["Ind"]
gdfs["density"] = gdfs["Ind"]/(0.0002)

In [ ]:
from pysal.model import spreg
from pysal.lib import weights
from spreg import OLS

In [ ]:
w_queen = weights.contiguity.Queen.from_dataframe(gdfs) # adjacent in all directions, including diagonal.
# Replace Queen by rook for only up/down/left/right adjancy

In [ ]:
w_queen.transform = 'r'

In [ ]:
dep_var = "CS_aggregated"
independ_var_B = ["%_soc.minimum", "%_>_65", "%_<_17", "%_<_bat_45", "%_>_bat_90", "%_residences", "mean_income","density"]
ols = OLS(gdfs[[dep_var]].values, gdfs[independ_var_B].values)

In [ ]:
sample = gdfs[independ_var_B]
sample.describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()
changevars = independ_var_B
gdfs[changevars] = scaler.fit_transform(gdfs[changevars])
gdfs["CS_aggregated"] += 0.0000001

In [ ]:
gdfs["CS_aggregated"] = scaler.fit_transform(gdfs[["CS_aggregated"]])

In [ ]:
mB = spreg.GM_Combo(
    gdfs[[dep_var]].values,
    gdfs[independ_var_B].values,
    w=w_queen,
    name_y=dep_var,
    name_x=independ_var_B,
)

print(mB.summary)